In [1]:
import pandas
import datetime
import os

In [30]:
baseDir = "../data/"
# Setup some col names
colList = []
for colNum in range(63):
    if colNum == 0: 
        colList.append( "data1" )
    elif colNum == 1: 
        colList.append( "AsyDateStr" )
    elif colNum == 62:
        colList.append( "hourly_average" )
    else:
        currMinute = str(colNum-2)
        if colNum-2 < 10:
            currMinute = "0" + str(colNum-2)
        colList.append( currMinute )

        
def split_asytype_datetime(row):
        currNoisyDtStr = row["AsyDateStr"].split("N6E01")[1]
        if "H" in row["AsyDateStr"]:
            currDateHourStr = "20" + currNoisyDtStr.split("H")[0] +\
                "-" +  currNoisyDtStr.split("H")[1][:2]
        else:
            currDateHourStr = "20" + currNoisyDtStr.split("D")[0] +\
                "-" +  currNoisyDtStr.split("D")[1][:2]
        return currDateHourStr   
    
def get_asy_type(row):
        if "ASY" in row["AsyDateStr"]:
            if "H" in row["AsyDateStr"]:
                return "AsyH"
            else:
                return "AsyD"
        else:
            if "H" in row["AsyDateStr"]:
                return "SymH"
            else:
                return "SymD"

rawAsyDFList = []        
# Loop through the directory and get all files
for root, dirs, files in os.walk(baseDir):
    for fName in files:
        if "asym" not in fName:
            continue
        print "currently working with-->", root + fName
        currAsyDF = pandas.read_csv( root + fName, delim_whitespace=True,\
                                    header=None, names=colList )
        currAsyDF["dateStr"] = currAsyDF.apply( split_asytype_datetime, axis=1 )
        currAsyDF["Asy_type"] = currAsyDF.apply( get_asy_type, axis=1 )
        rawAsyDFList.append( currAsyDF )
rawAsyDF = pandas.concat( rawAsyDFList )
# Convert the dataframe to
selColList = ["dateStr", "Asy_type"] + [ "0" + str(x) if x < 10 else str(x) for x in range(60) ]
rawAsyDF = rawAsyDF[selColList]
AsyDF = pandas.melt(rawAsyDF, id_vars=["dateStr", "Asy_type"], 
                  var_name="minute", value_name="Value")
AsyDF["datetimeStr"] = AsyDF["dateStr"] + "-" + AsyDF["minute"]
# AsyDF["date"] = pandas.to_datetime(AsyDF["datetimeStr"], format='%Y%m%d-%H-%M')
# AsyDF["hour"] = AsyDF["date"].apply(lambda x: x.strftime('%H'))
# AsyDF["minute"] = AsyDF["date"].apply(lambda x: x.strftime('%M'))
# AsyDF = AsyDF[ [ "date", "datetimeStr","Asy_type", "minute", "hour", "Value" ] ]
AsyDF = AsyDF[ [ "datetimeStr","Asy_type", "Value" ] ]
AsyDF = AsyDF.drop_duplicates()
AsyDF = AsyDF.pivot(index='datetimeStr',\
                columns='Asy_type', values='Value')
AsyDF.to_csv("../data/Asy_processed.txt", sep=' ')
AsyDF.head()

currently working with--> ../data/asym_2013.txt
currently working with--> ../data/asym_2014.txt
currently working with--> ../data/asym_2012.txt
currently working with--> ../data/asym_2011.txt


Asy_type,AsyD,AsyH,SymD,SymH
datetimeStr,,,,
20110101-00-00,17.0,13.0,1.0,-10.0
20110101-00-01,17.0,13.0,1.0,-10.0
20110101-00-02,17.0,13.0,1.0,-10.0
20110101-00-03,17.0,13.0,1.0,-10.0
20110101-00-04,17.0,13.0,1.0,-10.0
